In [1]:
!pip install invisible_watermark transformers safetensors
!pip install diffusers==0.18.0

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.3 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 13.4 MB/s eta 0:00:0000:0100:01
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.2 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.26.3
    Uninstalling diffusers-0.26.3:
      Successfully uninstalled diffusers-0.26.3


In [1]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained("./stable-diffusion-xl-base-1.0/", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")

# if using torch < 2.0
# pipe.enable_xformers_memory_efficient_attention()

The config attributes {'force_upcast': True} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
prompt = "An astronaut riding a green horse"
images = pipe(prompt=prompt).images[0]
print(images)

<PIL.Image.Image image mode=RGB size=1024x1024 at 0x7EFE877E5F00>


In [5]:
from PIL import Image

images.save("generate_example/1.png")

In [7]:
from diffusers import DiffusionPipeline
import torch


# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "./stable-diffusion-xl-base-1.0/", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "./stable-diffusion-xl-refiner-1.0/",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 40
high_noise_frac = 0.8

prompt = "A majestic lion jumping from a big stone at night"

# run both experts
image_base = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_start=high_noise_frac,
    image=image_base,
).images[0]
image.save("generate_example/refined.png")


The config attributes {'force_upcast': True} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
n_steps = 100
high_noise_frac = 0.8

prompt = "Jay Chou playing the piano in a concert, with realistic style"

# run both experts
image_base = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_start=high_noise_frac,
    image=image_base,
).images[0]
image.save("generate_example/person_real.png")

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [1]:
from diffusers import DiffusionPipeline
import torch


# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "./stable-diffusion-xl-base-1.0/", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda:1")
refiner = DiffusionPipeline.from_pretrained(
    "./stable-diffusion-xl-refiner-1.0/",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda:1")
n_steps = 50
high_noise_frac = 0.8

prompt = "Jay Chou in a fantasy world, playing piano, 8k resolution, 16:9 aspect ratio, 60fps, with a dreamy aesthetic."
image_base = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    # denoising_start=high_noise_frac,
    image=image_base,
).images[0]
image.save("generate_example/jay_real.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]